In [16]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from pytorch_forecasting import TimeSeriesDataSet
from pathlib import Path
import matplotlib.pyplot as plt
from helper_functions.engine import train
from sklearn.metrics import mean_absolute_error as mae
import numpy as np
from numpy import array, split
import pandas as pd
from copy import deepcopy as dc
import warnings
warnings.filterwarnings("ignore")

In [13]:
df = pd.read_csv("Datasets/Dataframes_one_ex_var/ModelBenchmark_dataframe.csv")
df.head(), df.shape

(                  Date  Price  FR Generation  FR Load
 0  2018-01-01 00:00:00   4.74        53625.0  56250.0
 1  2018-01-01 01:00:00   3.66        52398.0  54300.0
 2  2018-01-01 02:00:00   1.26        51825.0  53600.0
 3  2018-01-01 03:00:00 -20.10        50729.0  50000.0
 4  2018-01-01 04:00:00 -31.82        50719.0  47100.0,
 (43824, 4))

In [14]:
df = df[["Date", "Price"]]
df.head(2)

,Date,Price
0,2018-01-01 00:00:00,4.74
1,2018-01-01 01:00:00,3.66


In [15]:
df.Date = pd.to_datetime(df.Date)
df.set_index("Date", inplace=True)
df = df.astype("float32")
df.head(2)

,Price
Date,
2018-01-01 00:00:00,4.74
2018-01-01 01:00:00,3.66


In [26]:
# Define the test period
start_test_period = df.index.get_loc("2022-01-01").start
end_test_period = df.index.get_loc("2022-12-31").stop

43824

In [44]:
# split a univariate dataset into train/test sets
def split_dataset(df, start_test_period:int, end_test_period):
	# split into standard weeks
	train, test = df[0:start_test_period], df[start_test_period:end_test_period]
	# restructure into windows of daily data
	train = array(split(train, len(train)/24))
	test = array(split(test, len(test)/24))
	return train, test

# load the new file
train, test = split_dataset(df.values, start_test_period, end_test_period)
# validate train data
print(train.shape)
print(train[0, 0, 0], train[-1, -1, 0])
# validate test
print(test.shape)
print(test[0, 0, 0], test[-1, -1, 0])

(1461, 24, 1)
4.74 82.02
(365, 24, 1)
67.07 -4.39


In [50]:
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=24):
	"""
	train = list of days (history to be considered)
	n_input = number of time steps to use as inputs
	n_out = forecast horizon (here always 24 hour ahead)
	"""
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end <= len(data):
			x_input = data[in_start:in_end, 0]
			x_input = x_input.reshape((len(x_input), 1))
			X.append(x_input)
			y.append(data[in_end:out_end, 0])
		# move along one time step
		in_start += 1
	return array(X), array(y)

X_train, y_train = to_supervised(train, n_input=24*7)
X_train.shape, y_train.shape

((34873, 168, 1), (34873, 24))